In [1]:
import pandas as pd 
import pyodbc
import warnings
warnings.simplefilter('ignore')
import sqlite3


In [2]:

DB = {
    'servername': 'tcp:dwhunitedoutdoors.database.windows.net,1433',
    'database': 'DWH_United_Outdoors',
    'username': 'onderzoeker',
    'password': 'Wachtwoord1.' 
}

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=' + DB['servername'] + ';'
    'DATABASE=' + DB['database'] + ';'
    'UID=' + DB['username'] + ';'
    'PWD=' + DB['password'] + ';'
    'Encrypt=yes;'
    'TrustServerCertificate=no;'
    'Connection Timeout=30;'
)

export_conn = pyodbc.connect(conn_str)
export_cursor = export_conn.cursor()
export_cursor


In [5]:
DB_AW = {'servername': 'LAPTOPTK\\SQLEXPRESS',
'database': 'AdventureWorks2019'}

DB_AW_conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=' + DB_AW['servername'] +
                              ';DATABASE=' + DB_AW['database'] + ';Trusted_Connection=yes')


ProgrammingError: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot open database "AdventureWorks2019" requested by the login. The login failed. (4060) (SQLDriverConnect); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot open database "AdventureWorks2019" requested by the login. The login failed. (4060)')

In [ ]:
import pandas as pd
import pyodbc

# Establish the database connection
DB_AW_conn = pyodbc.connect("your_connection_string")
export_conn = pyodbc.connect("your_export_connection_string")

# Load data from the tables
newQuery = "SELECT * FROM [Sales].[Sales_SalesReason]"
sales_reason = pd.read_sql(newQuery, DB_AW_conn)

newQuery = "SELECT * FROM [Sales].[SalesOrderHeaderSalesReason]"
order_header_sales_reason = pd.read_sql(newQuery, DB_AW_conn)

newQuery = "SELECT * FROM [Sales].[SalesOrderHeader]"
order_header = pd.read_sql(newQuery, DB_AW_conn)

# Merge the DataFrames
merged_data = order_header.merge(order_header_sales_reason, on='SalesOrderID', how='left')
merged_data = merged_data.merge(sales_reason, on='SalesReasonID', how='left')

# Fill NaN values with appropriate defaults
merged_data['Name'].fillna('', inplace=True)
merged_data['ReasonType'].fillna('', inplace=True)

# Insert the merged data into a new table
export_cursor = export_conn.cursor()

for index, row in merged_data.iterrows():
    try:
        query = f"""
        INSERT INTO MergedSalesData (
            SalesOrderID, RevisionNumber, OrderDate, DueDate, ShipDate, Status,
            SalesOrderNumber, PurchaseOrderNumber, AccountNumber, CustomerID,
            SalesPersonID, TerritoryID, BillToAddressID, ShipToAddressID,
            ShipMethodID, CreditCardID, SubTotal, TaxAmt, TotalDue,
            SalesReasonID, Name, ReasonType
        ) VALUES (
            {row['SalesOrderID']}, {row['RevisionNumber']}, '{row['OrderDate']}', '{row['DueDate']}', 
            '{row['ShipDate']}', {row['Status']}, '{row['SalesOrderNumber']}', 
            '{row['PurchaseOrderNumber']}', '{row['AccountNumber']}', {row['CustomerID']}, 
            {row['SalesPersonID']}, {row['TerritoryID']}, {row['BillToAddressID']}, 
            {row['ShipToAddressID']}, {row['ShipMethodID']}, {row['CreditCardID']}, 
            {row['SubTotal']}, {row['TaxAmt']}, {row['TotalDue']}, {row['SalesReasonID']}, 
            '{row['Name']}', '{row['ReasonType']}'
        )
        """
        export_cursor.execute(query)
    except pyodbc.Error as e:
        print("An error occurred:", e)
        print(query)

export_conn.commit()
export_cursor.close()
